In [3]:
"""
Processing the data
"""
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler


def process_data(train, test, lags):
    """Process data
    Reshape and split train\test data.

    # Arguments
        train: String, name of .csv train file.
        test: String, name of .csv test file.
        lags: integer, time lag.
    # Returns
        X_train: ndarray.
        y_train: ndarray.
        X_test: ndarray.
        y_test: ndarray.
        scaler: StandardScaler.
    """
    attr = 'VFlow'
    df1 = pd.read_csv(train, encoding='utf-8').fillna(0)
    df2 = pd.read_csv(test, encoding='utf-8').fillna(0)

    # scaler = StandardScaler().fit(df1[attr].values)
    scaler = MinMaxScaler(feature_range=(0, 1)).fit(df1[attr].values.reshape(-1, 1))
    flow1 = scaler.transform(df1[attr].values.reshape(-1, 1)).reshape(1, -1)[0]
    flow2 = scaler.transform(df2[attr].values.reshape(-1, 1)).reshape(1, -1)[0]

    train, test = [], []
    for i in range(lags, len(flow1)):
        train.append(flow1[i - lags: i + 1])
    for i in range(lags, len(flow2)):
        test.append(flow2[i - lags: i + 1])

    train = np.array(train)
    test = np.array(test)
    np.random.shuffle(train)

    X_train = train[:, :-1]
    y_train = train[:, -1]
    X_test = test[:, :-1]
    y_test = test[:, -1]

    return X_train, y_train, X_test, y_test, scaler

In [4]:
def train_model(model, X_train, y_train, name, config):
    """train
    train a single model.

    # Arguments
        model: Model, NN model to train.
        X_train: ndarray(number, lags), Input data for train.
        y_train: ndarray(number, ), result data for train.
        name: String, name of model.
        config: Dict, parameter for train.
    """

    model.compile(loss="mse", optimizer="rmsprop", metrics=['mape'])
    # early = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='auto')
    hist = model.fit(
        X_train, y_train,
        batch_size=config["batch"],
        epochs=config["epochs"],
        validation_split=0.05)

    model.save('model/' + name + '.h5')
    df = pd.DataFrame.from_dict(hist.history)
    df.to_csv('model/' + name + ' loss.csv', encoding='utf-8', index=False)

In [5]:
import os

def do_train(model_name):
    lag = 12
    config = {"batch": 256, "epochs": 500}

    train_folder = 'intersection/train/'
    test_folder = 'intersection/test/'

    for filename in os.listdir(train_folder):
        if filename.endswith(".csv"):
            scats_number = filename.split('_')[1].split('.')[0]
            train_file = os.path.join(train_folder, filename)
            test_file = os.path.join(test_folder, f"test_{scats_number}.csv")

            print(f"Processing SCATS Number: {scats_number}")

            X_train, y_train, X_test, y_test, scaler = process_data(train_file, test_file, lag)

            if model_name == 'lstm':
                X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
                m = get_lstm([12, 64, 64, 1])
                train_model(m, X_train, y_train, model_name, config)

            elif model_name == 'gru':
                X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
                m = get_gru([12, 64, 64, 1])
                train_model(m, X_train, y_train, model_name, config)

            elif model_name == 'saes':
                X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
                m = get_saes([12, 400, 400, 400, 1])
                train_seas(m, X_train, y_train, model_name, config)

            print(f"Training complete for SCATS Number: {scats_number}")


In [6]:
"""
Defination of NN model
"""
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM, GRU
from keras.models import Sequential


def get_lstm(units):
    """LSTM(Long Short-Term Memory)
    Build LSTM Model.

    # Arguments
        units: List(int), number of input, output and hidden units.
    # Returns
        model: Model, nn model.
    """

    model = Sequential()
    model.add(LSTM(units[1], input_shape=(units[0], 1), return_sequences=True))
    model.add(LSTM(units[2]))
    model.add(Dropout(0.2))
    model.add(Dense(units[3], activation='sigmoid'))

    return model


def get_gru(units):
    """GRU(Gated Recurrent Unit)
    Build GRU Model.

    # Arguments
        units: List(int), number of input, output and hidden units.
    # Returns
        model: Model, nn model.
    """

    model = Sequential()
    model.add(GRU(units[1], input_shape=(units[0], 1), return_sequences=True))
    model.add(GRU(units[2]))
    model.add(Dropout(0.2))
    model.add(Dense(units[3], activation='sigmoid'))

    return model


def _get_sae(inputs, hidden, output):
    """SAE(Auto-Encoders)
    Build SAE Model.

    # Arguments
        inputs: Integer, number of input units.
        hidden: Integer, number of hidden units.
        output: Integer, number of output units.
    # Returns
        model: Model, nn model.
    """

    model = Sequential()
    model.add(Dense(hidden, input_dim=inputs, name='hidden'))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(output, activation='sigmoid'))

    return model


def get_saes(layers):
    """SAEs(Stacked Auto-Encoders)
    Build SAEs Model.

    # Arguments
        layers: List(int), number of input, output and hidden units.
    # Returns
        models: List(Model), List of SAE and SAEs.
    """
    sae1 = _get_sae(layers[0], layers[1], layers[-1])
    sae2 = _get_sae(layers[1], layers[2], layers[-1])
    sae3 = _get_sae(layers[2], layers[3], layers[-1])

    saes = Sequential()
    saes.add(Dense(layers[1], input_dim=layers[0], name='hidden1'))
    saes.add(Activation('sigmoid'))
    saes.add(Dense(layers[2], name='hidden2'))
    saes.add(Activation('sigmoid'))
    saes.add(Dense(layers[3], name='hidden3'))
    saes.add(Activation('sigmoid'))
    saes.add(Dropout(0.2))
    saes.add(Dense(layers[4], activation='sigmoid'))

    models = [sae1, sae2, sae3, saes]

    return models

In [7]:
def train_seas(models, X_train, y_train, name, config):
    """train
    train the SAEs model.

    # Arguments
        models: List, list of SAE model.
        X_train: ndarray(number, lags), Input data for train.
        y_train: ndarray(number, ), result data for train.
        name: String, name of model.
        config: Dict, parameter for train.
    """

    temp = X_train
    # early = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='auto')

    for i in range(len(models) - 1):
        if i > 0:
            p = models[i - 1]
            hidden_layer_model = Model(input=p.input,
                                       output=p.get_layer('hidden').output)
            temp = hidden_layer_model.predict(temp)

        m = models[i]
        m.compile(loss="mse", optimizer="rmsprop", metrics=['mape'])

        m.fit(temp, y_train, batch_size=config["batch"],
              epochs=config["epochs"],
              validation_split=0.05)

        models[i] = m

    saes = models[-1]
    for i in range(len(models) - 1):
        weights = models[i].get_layer('hidden').get_weights()
        saes.get_layer('hidden%d' % (i + 1)).set_weights(weights)

    train_model(saes, X_train, y_train, name, config)

## warning: this stuff will take ages to run and needs GPU time

In [8]:
from keras.models import Model
from keras.callbacks import EarlyStopping

In [9]:
do_train('saes')

Processing SCATS Number: 2000
Epoch 1/500


c:\Users\Ben\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0606 - mape: 306212.7188 - val_loss: 0.0386 - val_mape: 371.0653
Epoch 2/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0389 - mape: 367417.0625 - val_loss: 0.0250 - val_mape: 252.2790
Epoch 3/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0315 - mape: 415530.1875 - val_loss: 0.0205 - val_mape: 180.4790
Epoch 4/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0253 - mape: 427241.3438 - val_loss: 0.0162 - val_mape: 178.3480
Epoch 5/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0230 - mape: 220430.7812 - val_loss: 0.0149 - val_mape: 140.0742
Epoch 6/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0209 - mape: 367769.5312 - val_loss: 0.0142 - val_mape: 174.8112
Epoch 7/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0206 - mape: 196755.4375 - val_loss: 0.0148 - val_mape: 106.8335
Epoch 8/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0187 - mape: 224907.4375 - val_loss: 0.0117 - val_mape: 153.8294
Epoc

ValueError: The layer sequential has never been called and thus has no defined input.

In [10]:
do_train('gru')

Processing SCATS Number: 2000
Epoch 1/500


c:\Users\Ben\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 0.0571 - mape: 667727.3750 - val_loss: 0.0171 - val_mape: 292419.1875
Epoch 2/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0158 - mape: 78502.7578 - val_loss: 0.0136 - val_mape: 297358.2500
Epoch 3/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0126 - mape: 212039.6562 - val_loss: 0.0107 - val_mape: 301274.4688
Epoch 4/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0094 - mape: 224568.0469 - val_loss: 0.0083 - val_mape: 290113.2812
Epoch 5/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0077 - mape: 106468.3594 - val_loss: 0.0067 - val_mape: 252950.1250
Epoch 6/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0063 - mape: 218301.4375 - val_loss: 0.0059 - val_mape: 255793.3750
Epoch 7/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0058 - mape: 117023.6875 - val_loss: 0.0058 - val_mape: 247074.0469
Epoch 8/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0057 - mape: 136299.9375 - val_loss: 0.00

Training complete for SCATS Number: 2000
Processing SCATS Number: 2200
Epoch 1/500


c:\Users\Ben\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.0649 - mape: 3971120.2500 - val_loss: 0.0170 - val_mape: 3347033.7500
Epoch 2/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0173 - mape: 981114.2500 - val_loss: 0.0127 - val_mape: 2525845.7500
Epoch 3/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0136 - mape: 749520.6250 - val_loss: 0.0099 - val_mape: 2392059.5000
Epoch 4/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0110 - mape: 691234.8750 - val_loss: 0.0081 - val_mape: 2455713.2500
Epoch 5/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0089 - mape: 649873.0625 - val_loss: 0.0071 - val_mape: 2121961.2500
Epoch 6/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0076 - mape: 688901.0625 - val_loss: 0.0057 - val_mape: 2235378.7500
Epoch 7/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0067 - mape: 855620.1875 - val_loss: 0.0056 - val_mape: 2141421.0000
Epoch 8/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0064 - mape: 722915.8750 - val_l

KeyboardInterrupt: 

In [10]:
do_train('lstm')

Epoch 1/500


c:\Users\Ben\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.0719 - mape: 625751.5625 - val_loss: 0.0318 - val_mape: 191.2047
Epoch 2/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0291 - mape: 289894.8750 - val_loss: 0.0295 - val_mape: 152.8069
Epoch 3/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0234 - mape: 179886.4219 - val_loss: 0.0239 - val_mape: 156.8578
Epoch 4/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0197 - mape: 176446.3281 - val_loss: 0.0183 - val_mape: 158.1291
Epoch 5/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0149 - mape: 285369.3750 - val_loss: 0.0130 - val_mape: 158.6632
Epoch 6/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0118 - mape: 230626.7656 - val_loss: 0.0101 - val_mape: 146.0703
Epoch 7/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0107 - mape: 192326.5938 - val_loss: 0.0103 - val_mape: 120.9747
Epoch 8/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0102 - mape: 319584.9375 - val_loss: 0.0091 - val_mape: 113.0

## end of warning

In [11]:
import math
import warnings
import numpy as np
import pandas as pd
from keras.models import load_model
from tensorflow.keras.utils import plot_model
import sklearn.metrics as metrics
import matplotlib as mpl
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

from keras.src.legacy.saving import legacy_h5_format

In [12]:
def MAPE(y_true, y_pred):
    """Mean Absolute Percentage Error
    Calculate the mape.

    # Arguments
        y_true: List/ndarray, ture data.
        y_pred: List/ndarray, predicted data.
    # Returns
        mape: Double, result data for train.
    """

    y = [x for x in y_true if x > 0]
    y_pred = [y_pred[i] for i in range(len(y_true)) if y_true[i] > 0]

    num = len(y_pred)
    sums = 0

    for i in range(num):
        tmp = abs(y[i] - y_pred[i]) / y[i]
        sums += tmp

    mape = sums * (100 / num)

    return mape


def eva_regress(y_true, y_pred):
    """Evaluation
    evaluate the predicted resul.

    # Arguments
        y_true: List/ndarray, ture data.
        y_pred: List/ndarray, predicted data.
    """

    mape = MAPE(y_true, y_pred)
    vs = metrics.explained_variance_score(y_true, y_pred)
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)
    print('explained_variance_score:%f' % vs)
    print('mape:%f%%' % mape)
    print('mae:%f' % mae)
    print('mse:%f' % mse)
    print('rmse:%f' % math.sqrt(mse))
    print('r2:%f' % r2)

def plot_results(y_true, y_preds, names):
    """Plot
    Plot the true data and predicted data.

    # Arguments
        y_true: List/ndarray, ture data.
        y_pred: List/ndarray, predicted data.
        names: List, Method names.
    """
    d = '2016-3-4 00:00'
    x = pd.date_range(d, periods=96, freq='15min')

    fig = plt.figure()
    ax = fig.add_subplot(111)

    ax.plot(x, y_true, label='True Data')
    for name, y_pred in zip(names, y_preds):
        ax.plot(x, y_pred, label=name)

    plt.legend()
    plt.grid(True)
    plt.xlabel('Time of Day')
    plt.ylabel('Flow')

    date_format = mpl.dates.DateFormatter("%H:%M")
    ax.xaxis.set_major_formatter(date_format)
    fig.autofmt_xdate()


def main():
    lag = 12
    train_folder = 'intersection/train/'
    test_folder = 'intersection/test/'

    models = ['lstm', 'gru']
    names = ['LSTM', 'GRU']

    for filename in os.listdir(train_folder):
        if filename.endswith(".csv"):
            scats_number = filename.split('_')[1].split('.')[0]
            train_file = os.path.join(train_folder, filename)
            test_file = os.path.join(test_folder, f"test_{scats_number}.csv")

            print(f"Evaluating for SCATS Number: {scats_number}")
            _, _, X_test, y_test, scaler = process_data(train_file, test_file, lag)
            y_test = scaler.inverse_transform(y_test.reshape(-1, 1)).reshape(1, -1)[0]

            y_preds = []
            for model_name in models:
                model = load_model(f"model/{model_name}.h5")
                X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
                predicted = model.predict(X_test)
                predicted = scaler.inverse_transform(predicted.reshape(-1, 1)).reshape(1, -1)[0]
                y_preds.append(predicted[:96])

                print(f"Evaluating {model_name} for SCATS Number: {scats_number}")
                eva_regress(y_test, predicted)

            plot_results(y_test[:96], y_preds, names)


In [13]:
main()

Evaluating for SCATS Number: 2000


TypeError: Could not locate function 'mse'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'keras.metrics', 'class_name': 'function', 'config': 'mse', 'registered_name': 'mse'}